# 02 - Detect Model Bias

Let's first import our dataset and pre-process it:

In [3]:
import pandas as pd

# Load dataset into dataframe
loan_dataset = pd.read_csv("../datasets/loan.csv")
loan_dataset.head()

# Prepare categorical and numeric features
categorical_features = ["sex", "rent", "minority", "ZIP", "occupation"]
numeric_features = [
    "education", "age", "income", "loan_size", "payment_timing",
    "year", "job_stability"
]
for cat in categorical_features:
    loan_dataset[cat] = loan_dataset[cat].astype("object")

We first need to define which variable is going to be our __outcome variable__ (the one we want to predict), and which are going to be our __sensitive features__ (those that the modeler should take into account when evaluating the fairness of the data or algorithm).

In [4]:
# Define outcome variable:
pred = "default"

# Define sensitive features:
sensitive_features = ["minority", "sex"]

Let's know get our data into the right format to be the input of a machine learning algorithms in scikit-learn:

In [5]:
# Define X and y
X = loan_dataset.copy().drop([pred], axis=1)
y = (loan_dataset.copy()[pred] != f"{pred}-no").astype(int).values

We can now run a __logistic regression__ to predict our outcome variable:

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Create preprocessor of features
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())]
)

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Create pipeline
clf = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression())
    ]
)

# Split into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

# Train classifier
clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['education', 'age', 'income',
                                                   'loan_size',
                                                   'payment_timing', 'year',
                                                   'job_stability']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['sex', 'rent', 'minority',
                                                   'ZIP', 'occupation'])])),
                ('classifier', LogisticRegression())])

Let's use our models to make predictions on the whole dataset:

In [8]:
# Predict
y_pred = clf.predict(X)
print(f"Example of the first twenty predictions: {y_pred[:20]}")

Example of the first twenty predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


# Quality of service harm

Let's inspect how the accuracy of the model changes for the different __sensitive subpopulations__ defined by the __sensitive features__:

In [14]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import recall_score, precision_score 

# Break precision and recall for different subpopulations
for sf in sensitive_features:
    grouped_metric = MetricFrame(
        {"precision": precision_score, "recall": recall_score}, y, y_pred,
        sensitive_features=loan_dataset["minority"]
    )
    grouped_metric_df = grouped_metric.by_group
    display(grouped_metric_df)

print(f"Overall precision and recall:")
display(pd.DataFrame(grouped_metric.overall, columns=["Overall accuracy"]))

,precision,recall
minority,,
minority-no,1.0,1.0
minority-yes,0.999695,1.0


,precision,recall
minority,,
minority-no,1.0,1.0
minority-yes,0.999695,1.0


Overall precision and recall:


,Overall accuracy
precision,0.999696
recall,1.0


# Quality of allocation harm

Let's know inspect which values of `default` get predicted for each sensitive subpopulation. We will print the propotion that was assigned to each label:

In [16]:
for sf in sensitive_features:
    print(f"Sensitive feature: {sf}")
    pred_grouped = pd.DataFrame({f"{sf}": loan_dataset[sf], "y_pred": y_pred, "y_true": y})
    pred_vals = pred_grouped.groupby(sf).sum().values / loan_dataset[sf].value_counts().values
    pred_grouped = pd.DataFrame(pred_vals, columns=[f"{pred}_predicted", f"{pred}_true"])
    display(pred_grouped)

Sensitive feature: minority


,default_predicted,default_true
0,0.001453,0.001455
1,0.997851,0.998937


Sensitive feature: sex


,default_predicted,default_true
0,0.5,0.499696
1,0.5,0.500000
